In [ ]:
import time
import numpy as np
from scipy.optimize import milp
from scipy.optimize import LinearConstraint


start_time = time.time()

## DIAGONAL MATRIX FUNCTION DEFINITION
def diagonal_identity(n, i):
    # Create a zero matrix of shape (n, i)
    matrix = np.zeros((n, i))
    # Fill the diagonal with ones
    min_dim = min(n, i)
    np.fill_diagonal(matrix, 1)
    return matrix


## DATA IMPORT -------------------------------------------------------------------------------------------------------------------
data_dir = '../data/raw/'
filename = 'g14.txt'
import_dir = data_dir + filename

# Load the data regarding points connected with edges (only first 2 columns are needed)
data_import = np.loadtxt(import_dir, usecols=(0, 1), dtype=int)

# Get total number of individual points from first row of dataset
npoints = data_import[0, 0]
# Get total number of individual weighted edges from first row of dataset
nedges = data_import[0, 1]

# Delete the first row (index 0) of the data, since relevant data has been stored
data = np.delete(data_import, 0, axis=0)

# Import edge weights from 3rd column
w = np.loadtxt(import_dir, usecols=(2), skiprows=1, dtype=int)

## INEQUALITY EQUATIONS ----------------------------------------------------------------------------------------------------------
# Define the [A] matrix dimensions of each equation set
rows = nedges                # Number of rows = number of entries in the file
cols = npoints + nedges      # Number of columns (based on max value in data)

# Pre-allocate the [A] matrix for both equation sets, using [0] matrices
matrix1 = np.zeros((rows, cols), dtype=int)
matrix2 = np.zeros((rows, cols), dtype=int)

# Matrix 3 => -xi <= 0
matrix3 = (-1)*diagonal_identity(cols, cols)

# Matrix 4 => xi <= 1
matrix4 = diagonal_identity(cols,cols)

# Fill the matrix with 1s at the specified positions, according to relevant eqns.
for i, (col1, col2) in enumerate(data):
    # Equation 1:
    matrix1[i, col1] = -1                  # xi variable coefficient
    matrix1[i, col2] = -1                  # xj variable coefficient
    matrix1[i, i + npoints] = 1            # eij variable coefficient

    # Equation 2:
    matrix2[i, col1] = 1                   
    matrix2[i, col2] = 1
    matrix2[i, i + npoints] = 1

# Assemble [A] matrix by stacking [A1] and [A2]
A = np.vstack((matrix1, matrix2, matrix3, matrix4))

# Right-hand side of inequality corresponds to {b} vector
b1 = np.zeros(rows, dtype=int)  # Now shape is (N,)
b2 = np.full(rows, 2, dtype=int)  # Also (N,)

b3 = np.zeros(cols, dtype=int)
b4 = np.full(cols, 1, dtype=int)

# Assemble {b} vector by stacking {b1} and {b2}
b_u = np.hstack((b1, b2, b3, b4))  # Result is already 1D
b_l = np.full_like(b_u, -np.inf, dtype=float)

## MAXIMIZING FUNCTION ----------------------------------------------------------------------------------------------------------
# We now describe the function which we wish to maximize. This function corresponds to the sum of
# all cut edges (e_ij ~= 0) multiplied by their respective weight (w_ij)

# Coefficients for all x_i variables are zero, since solution depends only on edges
c1 = np.zeros(npoints, dtype=int)

# Coefficients for all e_ij variables are their respective weight values, which are imported from
# the given dataset
c2 = w

# Assemble {c} vector by stacking {c1} and {c2}
c = np.hstack((c1, c2))

# Since we want to maximize, we minimize the negative of the objective function
c = -c

## BOUNDING VARIABLES ----------------------------------------------------------------------------------------------------------
# Define bounds for the variables (0 <= x <= 1)

constraints = LinearConstraint(A, b_l, b_u)

# Define integer constraints for the variables
integrality = [1] * npoints + [1] * nedges  

## LINEAR PROGRAMMING SOLUTION ---------------------------------------------------------------------------------------------------
# Solve the mixed-integer linear programming problem
res = milp(c=c, constraints=constraints, integrality=integrality)

## OUTPUTS ----------------------------------------------------------------------------------------------------------------------
if res.success:
    print("Optimal values:", res.x)
    print("Optimal objective function value:", -res.fun)  
else:
    print("Optimization failed:", res.message)

# Generate output matrix
point_groups = res.x[:npoints]
points = np.arange(1, npoints + 1)
output = np.column_stack((points, point_groups))

# Write to file
output_name = f'output_{filename}'

# Save output to a text file
np.savetxt(output_name, output, fmt='%.6f')


# End the timer
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time

print(f"Optimization complete. Results saved to: {output_name}")
print(f"Elapsed time: {elapsed_time:.2f} seconds")